# README: How to run this notebook

The whole jupyter notebook could be seperated into several parts.

The first part is basic library import and model configuration, including wandb initialization, log set up and the finish of creation of dictionary and the function `transform_index_to_letter`.For hyperparameter setting, batch size 

The next part is to finish the import of dataset and create the dataloader for model training. In this part, I included the cepstral normalization, which could make greate contributation towards the convergence. 

The following is encoder part, which i implemented the pblstm and lockedDropout with reference. Then, I used the torch summary to validate the function of this model. 

The next part is attention, where i initially implemented the single head attention (without scaled version), then I switched to multihead, with head equals four suggested by TAs, but in my case, head size set to two achieves best. 

Then it follows the decoding part, in which teaching force and two fully connected layer with tanh suggested. I tried other activation function, but tanh achieves the best. Also, layer initlization methods were also included. I also tried add more lstm cells/layers (2->3) in decoding part, but the performance don't improve. I also used a torch summary to validate the correct implementation of the whole seq2seq model. In addition, as suggested by my classmates, I could transform the key value of multihead attention in seq2seq model to avoid repeated calculation .

For the training part, I used an warm up schedular with cosine annealing to warm up the whole training process, after the 100 epochs, I could achieve 9.2 in LD on test data on kaggle. 

All my ablation tests could be viewed in wandb
https://wandb.ai/saltedfish/11785-HW4?workspace=user-saltedfish


In [ ]:
!nvidia-smi

Fri Apr 22 00:45:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    42W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: www-browser: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links2: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: elinks: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: lynx: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.

In [ ]:
import json

TOKEN = {"username":"yuxuanwucmu","key":"81ded4babfd327efdc7655be369299b5"}

!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
! mkdir -p .kaggle
! mkdir -p /content & mkdir -p /content/.kaggle & mkdir -p /root/.kaggle/

with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(TOKEN, file)

! pip install --upgrade --force-reinstall --no-deps kaggle
! ls "/content/.kaggle"
! chmod 600 /content/.kaggle/kaggle.json
! cp /content/.kaggle/kaggle.json /root/.kaggle/

! kaggle config set -n path -v /content/drive/MyDrive/

! kaggle competitions download -c 11-785-s22-hw4p2
! unzip -q /content/drive/MyDrive/competitions/11-785-s22-hw4p2/11-785-s22-hw4p2.zip

!pip install python-levenshtein

     |████████████████████████████████| 59 kB 3.5 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73275 sha256=bbab5ec351ebf49c4a7f586db40ea2c2227eab74fa891aa2c0baad8dc350da5c
  Stored in directory: /root/.cache/pip/wheels/de/f7/d8/c3902cacb7e62cb611b1ad343d7cc07f42f7eb76ae3a52f3d1
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
     |████████████████████████████████| 58 kB 3.5 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=a11d3cb97d6ca3dfc8d469e0b3dac00757f6fade5a69354a07ae6d77c224c228
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8
kaggle.json
- 

# Libraries and Initial Processing

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import Levenshtein as lev
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio.transforms as T
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.utils as utils
import seaborn as sns
import matplotlib.pyplot as plt
import time
import random
import datetime
from torch.utils import data
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import csv
! pip install torchsummaryX
from torchsummaryX import summary
!pip install wandb
import wandb
from tqdm import tqdm
!pip install pytorch_warmup
import pytorch_warmup as warmup
def create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)

cuda = torch.cuda.is_available()

print(cuda, sys.version)

device = torch.device("cuda" if cuda else "cpu")
num_workers = 4 if cuda else 0
print("Cuda = "+str(cuda)+" with num_workers = "+str(num_workers))
np.random.seed(11785)
torch.manual_seed(11785)

# The labels of the dataset contain letters in LETTER_LIST.
# You should use this to convert the letters to the corresponding indices
# and train your model with numerical labels.
LETTER_LIST = ['<sos>', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', \
         'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', "'", ' ', '<eos>']

     |████████████████████████████████| 1.8 MB 5.1 MB/s 
     |████████████████████████████████| 144 kB 76.9 MB/s 
     |████████████████████████████████| 181 kB 85.2 MB/s 
     |████████████████████████████████| 63 kB 2.5 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=747d2b9a11f010716cf4ed4c506b655bed8d5c374ff02341c9d1cab75e27b2e5
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
     |████████████████████████████████| 314 kB 5.1 MB/s 
  Created wheel for pytorch-warmup: filename=pytorch_warmup-0.1.0-py3-none-any.whl size=5802 sha256=be559fa079fd5f121db1123d8994388012915a907e4fbf0d5c8dfc6d5b43a98c
  Stored in directory: /root/.cache/pip/wheels/c7/5d/24/9475e442daa1e9332c122c79fb5131b9e4e91946009365902e
Successfully built pytorch-warmup
True 3.7.13 (default, Mar 16 2022, 17:37:17) 
[GCC 7.5.0]
Cuda = True with num_workers = 4


# Model Configuration

In [ ]:
checkpoints_store_path = "/content/drive/MyDrive/hw4/store_checkpoints/"
create_folder(checkpoints_store_path)

n_epochs = 100
batch_size = 128
lr = 1e-3
normalization = "Cepstral"
tf = 0.9 
tf_sep_epoch = 5
warmup_type = "radam"
init = "embedding_init"
dropoutVal = 0.2

model_id = f"Freq_Aug_2_Multiheaded_Masking_AdamW_SameKeyValueIncreseTest_{normalization}_tf_{tf}_sep{tf_sep_epoch}_{init}_warmup_{warmup_type}_lockeddropout_{dropoutVal}"
data_path = checkpoints_store_path + model_id
create_folder(data_path)


wandb.init(project="11785-HW4", entity="saltedfish",name=model_id)
wandb.config = {
  "learning_rate": lr,
  "epochs": n_epochs,
  "batch_size": batch_size,
  "tf":tf,
  "init": init
}


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
def create_dictionaries(letter_list):
    '''
    Create dictionaries for letter2index and index2letter transformations
    based on LETTER_LIST

    Args:
        letter_list: LETTER_LIST

    Return:
        letter2index: Dictionary mapping from letters to indices
        index2letter: Dictionary mapping from indices to letters
    '''
    letter2index = dict()
    index2letter = dict()
    
    for i in range(len(LETTER_LIST)):
        letter2index[LETTER_LIST[i]] = i
        index2letter[i] = LETTER_LIST[i]

    return letter2index, index2letter
    

def transform_index_to_letter(batch_indices):
    '''
    Transforms numerical index input to string output by converting each index 
    to its corresponding letter from LETTER_LIST

    Args:
        batch_indices: List of indices from LETTER_LIST with the shape of (N, )
    
    Return:
        transcripts: List of converted string transcripts. This would be a list with a length of N
    '''
    transcripts = []
    start_index = letter2index['<sos>']
    stop_index = letter2index['<eos>']

    for tr in batch_indices:
      curr = ""
      for i in tr:
        if i == stop_index:
          break
        elif i == start_index:
          pass
        else:
          curr += index2letter[i]
      transcripts.append(curr)

    # TODO
    return transcripts

# Create the letter2index and index2letter dictionary
letter2index, index2letter = create_dictionaries(LETTER_LIST)
print(letter2index)
print(index2letter)



{'<sos>': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26, "'": 27, ' ': 28, '<eos>': 29}
{0: '<sos>', 1: 'A', 2: 'B', 3: 'C', 4: 'D', 5: 'E', 6: 'F', 7: 'G', 8: 'H', 9: 'I', 10: 'J', 11: 'K', 12: 'L', 13: 'M', 14: 'N', 15: 'O', 16: 'P', 17: 'Q', 18: 'R', 19: 'S', 20: 'T', 21: 'U', 22: 'V', 23: 'W', 24: 'X', 25: 'Y', 26: 'Z', 27: "'", 28: ' ', 29: '<eos>'}


# Dataset and Dataloading

You will need to implement the Dataset class by your own. You can implement it similar to HW3P2. However, you are welcomed to do it your own way if it is more comfortable or efficient.

Note that you need to use LETTER_LIST to convert the transcript into numerical labels for the model.


Example of raw transcript:

    ['<sos>', 'N', 'O', 'R', 'T', 'H', 'A', 'N', 'G', 'E', 'R', ' ','A', 'B', 'B', 'E', 'Y', '<eos>']

Example of converted transcript ready to process for the model:

    [0, 14, 15, 18, 20, 8, 1, 14, 7, 5, 18, 28, 1, 2, 2, 5, 25, 29]


In [ ]:
class LibriSamples(torch.utils.data.Dataset):

    def __init__(self, data_path, partition= "train"):
        self.X_dir = data_path + "/" + partition + "/mfcc/"
        self.Y_dir = data_path + "/" + partition + "/transcript/"

        self.X_files = os.listdir(self.X_dir)
        self.Y_files = os.listdir(self.Y_dir)


    def __len__(self):
        return len(self.X_files)

    def __getitem__(self, ind):
        X = np.load(self.X_dir + self.X_files[ind])
        X = (X - X.mean(axis=0)) / X.std(axis=0)
        Y = [letter2index[yy] for yy in np.load(self.Y_dir + self.X_files[ind])]  # TODO: Load the corresponding index
        # print(Y)
        Yy = torch.LongTensor(Y)

        return X, Yy
    
    def collate_fn(batch):
        batch_x = [x for (x, y) in batch]
        batch_y = [y for (x, y) in batch]

        batch_x_pad = pad_sequence([torch.Tensor(x) for x in batch_x],batch_first=True)  # TODO: pad the sequence with pad_sequence (already imported)
        lengths_x = torch.LongTensor([len(x) for x in batch_x])  # TODO: Get original lengths of the sequence before padding

        batch_y_pad = pad_sequence(batch_y,batch_first=True,padding_value=0)  # TODO: pad the sequence with pad_sequence (already imported)
        lengths_y = torch.LongTensor([len(y) for y in batch_y])   # TODO: Get original lengths of the sequence before padding

        return batch_x_pad, batch_y_pad, torch.tensor(lengths_x), torch.tensor(lengths_y)


class LibriSamplesTest(torch.utils.data.Dataset):

    def __init__(self, data_path, test_order):  # test_order is the csv similar to what you used in hw1
        file_path = data_path + '/test/' + test_order
        self.X_dir = data_path + "/test/mfcc/"

        test_order_list = self.parse_csv(file_path)  # TODO: open test_order.csv as a list
        self.X = [i for i in test_order_list]  # TODO: Load the npy files from test_order.csv and append into a list
        # You can load the files here or save the paths here and load inside __getitem__ like the previous class

    def __len__(self):
        return len(self.X)

    def __getitem__(self, ind):
        # TODOs: Need to return only X because this is the test dataset
        # print(self.X[ind])
        # print(self.X_dir+self.X[ind][0])
        X = np.load(self.X_dir+self.X[ind][0])
        X = (X - X.mean(axis=0)) / X.std(axis=0)
        return X

    def collate_fn(batch):
        batch_x_pad = pad_sequence(
            [torch.Tensor(x) for x in batch],batch_first=True)  # TODO: pad the sequence with pad_sequence (already imported)
        lengths_x = torch.LongTensor(
            [len(x) for x in batch])  # TODO: Get original lengths of the sequence before padding

        return batch_x_pad, torch.tensor(lengths_x)

    @staticmethod
    def parse_csv(filepath):
        subset = []
        with open(filepath) as f:
            f_csv = csv.reader(f)
            for row in f_csv:
                subset.append(row)
        return subset[1:]

In [ ]:
root = '/hw4p2_student_data/hw4p2_student_data'

train_data = LibriSamples(root, 'train')
val_data = LibriSamples(root, 'dev')
test_data = LibriSamplesTest(root, 'test_order.csv')

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True,
                                           collate_fn=LibriSamples.collate_fn)  # TODO: Define the train loader. Remember to pass in a parameter (function) for the collate_fn argument
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True,
                                         collate_fn=LibriSamples.collate_fn)  # TODO: Define the val loader. Remember to pass in a parameter (function) for the collate_fn argument
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False,
                                          collate_fn=LibriSamplesTest.collate_fn)  # TODO: Define the test loader. Remember to pass in a parameter (function) for the collate_fn argument

print("Batch size: ", batch_size)
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  128
Train dataset samples = 28539, batches = 223
Val dataset samples = 2703, batches = 22
Test dataset samples = 2620, batches = 21


In [ ]:
# test code for checking shapes
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, len(ly))
    print(y[0]) # desired 
    # print(x[0].shape)
    # print(x[0,:,:].shape)
    break

# Test code for checking shapes and return arguments of the train and val loaders
for data in test_loader:
    x, lx = data # if you face an error saying "Cannot unpack", then you are not passing the collate_fn argument
    # print("===================")
    # print(data)
    # print("===================")
    print(x.shape, lx.shape)
    break

torch.Size([128, 1678, 13]) torch.Size([128, 330]) torch.Size([128]) 128
tensor([ 0,  9, 28, 23,  1, 19, 28,  2,  5,  7,  9, 14, 14,  9, 14,  7, 28, 20,
        15, 28, 20,  1, 11,  5, 28,  1, 14, 28,  9, 14, 20,  5, 18,  5, 19, 20,
        28,  9, 14, 28, 10,  9, 13, 28, 19, 15, 28,  9, 28,  2, 18, 15, 21,  7,
         8, 20, 28,  8,  9, 13, 28, 21, 16, 28,  9, 14, 20, 15, 28, 20,  8,  5,
        28, 15,  6,  6,  9,  3,  5, 28,  1, 14,  4, 28, 19,  5, 20, 28,  8,  9,
        13, 28, 20, 15, 28,  3, 15, 16, 25,  9, 14,  7, 28,  3,  9, 18,  3, 21,
        12,  1, 18, 28, 12,  5, 20, 20,  5, 18, 19, 28, 23,  5, 28, 21, 19,  5,
         4, 28, 20, 15, 28, 19,  5, 14,  4, 28, 15, 21, 20, 28,  1, 28, 18,  1,
         6, 20, 28, 15,  6, 28, 20,  8,  5, 13, 28, 20, 15, 28, 20,  8,  5, 28,
        20, 18,  1,  4,  5, 28, 20,  8,  1, 20, 28, 23,  1, 19, 28, 10, 21, 19,
        20, 28,  2,  5,  6, 15, 18,  5, 28, 20,  8,  5, 28,  7,  5, 14,  5, 18,
         1, 12, 28,  1,  4, 15, 16, 20,  9, 15,

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [ ]:
# test code for checking shapes
for data in test_loader:
    x, lx = data
    print(x.shape, lx.shape)
    print(x[0].shape) # desired 
    # print(x[0,:,:])
    # print(x[0,:,:].shape)
    break

torch.Size([128, 3003, 13]) torch.Size([128])
torch.Size([3003, 13])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


# Model (Encoder)

In [ ]:
# Lockdropout
# Code from TA and two externel links
# https://github.com/Lobbii/CMU-11785-Homework/blob/main/Attention-based%20End-to-End%20Speech-to-Text%20Deep%20Neural%20Network/HW4P2_template_Final.ipynb
# https://pytorchnlp.readthedocs.io/en/latest/_modules/torchnlp/nn/lock_dropout.html
from torch.autograd import Variable

class LockedDropout(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, x, dropout=0.2):
    """
    :param x: (B,T,C)
    :param p: probability of lockedDropout val 
    :return:
    """

    if dropout == 0 or not self.training:
      return x
    mask = x.data.new(x.size(0), 1, x.size(2))
    mask = mask.bernoulli_(1 - dropout)
    mask = Variable(mask, requires_grad=False) / (1 - dropout)
    mask = mask.expand_as(x)
    return mask * xty

In [ ]:

class pBLSTM(nn.Module):
    '''
    Pyramidal BiLSTM
    Read paper and understand the concepts and then write your implementation here.

    At each step,
    1. Pad your input if it is packed
    2. Truncate the input length dimension by concatenating feature dimension
        (i) How should  you deal with odd/even length input? 
        (ii) How should you deal with input length array (x_lens) after truncating the input?
    3. Pack your input
    4. Pass it into LSTM layer

    To make our implementation modular, we pass 1 layer at a time.
    '''
    def __init__(self, input_dim, hidden_dim):
        super(pBLSTM, self).__init__()
        self.blstm = nn.LSTM(input_size=input_dim*2,hidden_size=hidden_dim, num_layers=1,bidirectional=True,batch_first=True)
        self.dropout = LockedDropout()

    def forward(self, x):

        #  1. Pad your input if it is packed # (B,T,C)
        #  x is a pack_padded_sequence unpack the LSTM output using pad_packed_sequence
        # print(x)
        x, x_lens = pad_packed_sequence(x,batch_first=True)

        B,T,C = x.shape
        # 2. Truncate the input length dimension by concatenating feature dimension
        #     (i) How should  you deal with odd/even length input? 
        if T%2==1: 
            x = x[:,:-1:] # remove the last one

        #     (ii) How should you deal with input length array (x_lens) after truncating the input?
        x_lens = x_lens // 2

        x = x.contiguous().reshape(B,T//2, C*2)

        x = self.dropout(x) # lock dropout layer

        # 3. Pack your input
        x = pack_padded_sequence(x,x_lens,batch_first=True,enforce_sorted=False)
        out = self.blstm(x)[0]
        return out


In [ ]:
class Encoder(nn.Module):
    '''
    Encoder takes the utterances as inputs and returns the key, value and unpacked_x_len.

    '''
    def __init__(self, input_dim, encoder_hidden_dim, key_value_size=128):
        super(Encoder, self).__init__()
        # The first LSTM layer at the bottom
        self.lstm = nn.LSTM(input_size=input_dim,hidden_size=encoder_hidden_dim,batch_first=True,bidirectional=True)

        # Define the blocks of pBLSTMs
        # Dimensions should be chosen carefully
        # Hint: Bidirectionality, truncation...

        # stack 3 pBLSTMs on top of the bottom BLSTM layer to reduce the time resolution 2^3 = 8 times
        self.pBLSTMs = nn.Sequential(
            pBLSTM(encoder_hidden_dim*2,encoder_hidden_dim),
            pBLSTM(encoder_hidden_dim*2,encoder_hidden_dim),
            pBLSTM(encoder_hidden_dim*2,encoder_hidden_dim),
            # Optional: dropout
        )
         
        # The linear transformations for producing Key and Value for attention
        # Hint: Dimensions when bidirectional lstm? 
        self.key_network = nn.Linear(in_features=encoder_hidden_dim*2,out_features=key_value_size)
        self.value_network = nn.Linear(in_features=encoder_hidden_dim*2,out_features=key_value_size)



    def forward(self, x, x_len):
        """
        input x.shape(B,T,C) x[0] = (128,3204,13)
        1. Pack your input and pass it through the first LSTM layer (no truncation)
        2. Pass it through the pyramidal LSTM layer
        3. Pad your input back to (B, T, *) or (T, B, *) shape
        4. Output Key, Value, and truncated input lens

        Key and value could be
            (i) Concatenated hidden vectors from all time steps (key == value).
            (ii) Linear projections of the output from the last pBLSTM network.
                If you choose this way, you can use the final output of
                your pBLSTM network.
        """
        # 1. Pack your input and pass it through the first LSTM layer (no truncation)
        # print("x.shape",x.shape)
        # print("x_lens.shape",x_len.shape)
        # print("x_lens",x_len)
        packed_input = pack_padded_sequence(x,x_len.cpu(), enforce_sorted=False,batch_first =True) #(B,T,C) 

        x = self.lstm(packed_input)[0] 
        # 2. Pass it through the pyramidal LSTM layer
        x = self.pBLSTMs(x)
        out, lens =  pad_packed_sequence(x,batch_first=True) 
        k = self.key_network(out)
        v = self.value_network(out)
        # print(out.shape)
        # print(k.shape)
        # print(v.shape)
        # print(lens)

        return k, v, lens

In [ ]:
encoder = Encoder(13,128).to(device)
summary(encoder, x.to(device), lx) # x and lx are from the previous cell
# print(encoder)
# print(x.shape)
# print(lx.shape)
# Try out your encoder on a tiny input before moving to the next step...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
/usr/local/lib/python3.7/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


                                  Kernel Shape      Output Shape    Params  \
Layer                                                                        
0_lstm                                       -      [72304, 256]  146.432k   
1_pBLSTMs.0.LockedDropout_dropout            -  [128, 1501, 512]         -   
2_pBLSTMs.0.LSTM_blstm                       -      [36124, 256]  657.408k   
3_pBLSTMs.1.LockedDropout_dropout            -   [128, 750, 512]         -   
4_pBLSTMs.1.LSTM_blstm                       -      [18034, 256]  657.408k   
5_pBLSTMs.2.LockedDropout_dropout            -   [128, 375, 512]         -   
6_pBLSTMs.2.LSTM_blstm                       -       [8987, 256]  657.408k   
7_key_network                       [256, 128]   [128, 375, 128]   32.896k   
8_value_network                     [256, 128]   [128, 375, 128]   32.896k   

                                  Mult-Adds  
Layer                                        
0_lstm                             144.384k  
1_p

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_lstm,-,"[72304, 256]",146432.0,144384.0
1_pBLSTMs.0.LockedDropout_dropout,-,"[128, 1501, 512]",NaN,NaN
2_pBLSTMs.0.LSTM_blstm,-,"[36124, 256]",657408.0,655360.0
3_pBLSTMs.1.LockedDropout_dropout,-,"[128, 750, 512]",NaN,NaN
4_pBLSTMs.1.LSTM_blstm,-,"[18034, 256]",657408.0,655360.0
5_pBLSTMs.2.LockedDropout_dropout,-,"[128, 375, 512]",NaN,NaN
6_pBLSTMs.2.LSTM_blstm,-,"[8987, 256]",657408.0,655360.0
7_key_network,"[256, 128]","[128, 375, 128]",32896.0,32768.0
8_value_network,"[256, 128]","[128, 375, 128]",32896.0,32768.0


# Attention 

In [ ]:
def plot_attention(attention):
    # utility function for debugging
    plt.clf()
    sns.heatmap(attention, cmap='GnBu')
    plt.show()

class Attention(nn.Module):
    '''
    Attention is calculated using key and value from encoder and query from decoder.
    Here are different ways to compute attention and context:
    1. Dot-product attention
        energy = bmm(key, query) 
        # Optional: Scaled dot-product by normalizing with sqrt key dimension
        # Check "attention is all you need" Section 3.2.1
    * 1st way is what most TAs are comfortable with, but if you want to explore...
    2. Cosine attention
        energy = cosine(query, key) # almost the same as dot-product xD 
    3. Bi-linear attention
        W = Linear transformation (learnable parameter): d_k -> d_q
        energy = bmm(key @ W, query)
    4. Multi-layer perceptron
        # Check "Neural Machine Translation and Sequence-to-sequence Models: A Tutorial" Section 8.4
    
    After obtaining unnormalized attention weights (energy), compute and return attention and context, i.e.,
    energy = mask(energy) # mask out padded elements with big negative number (e.g. -1e9)
    attention = softmax(energy)
    context = bmm(attention, value)

    5. Multi-Head Attention
        # Check "attention is all you need" Section 3.2.2
        h = Number of heads
        W_Q, W_K, W_V: Weight matrix for Q, K, V (h of them in total)
        W_O: d_v -> d_v

        Reshape K: (B, T, d_k)
        to (B, T, h, d_k // h) and transpose to (B, h, T, d_k // h)
        Reshape V: (B, T, d_v)
        to (B, T, h, d_v // h) and transpose to (B, h, T, d_v // h)
        Reshape Q: (B, d_q)
        to (B, h, d_q // h)

        energy = Q @ K^T
        energy = mask(energy)
        attention = softmax(energy)
        multi_head = attention @ V
        multi_head = multi_head reshaped to (B, d_v)
        context = multi_head @ W_O
    '''
    def __init__(self):
        super(Attention, self).__init__()
        # Optional: dropout

    def forward(self, query, key, value, mask):
        """
        input:
            key: (batch_size, seq_len, d_k)
            value: (batch_size, seq_len, d_v)
            query: (batch_size, d_q)
        * Hint: d_k == d_v == d_q is often true if you use linear projections
        return:
            context: (batch_size, key_val_dim)
        
        """
        key,mask = key.cuda(),mask.cuda()
        # (B,T,K_dim) mat-mat product (B, Q_dim,1)
        energy = torch.bmm(key,query.unsqueeze(2)).squeeze(2).cuda()

        energy = energy.masked_fill(mask,float("-inf"))

        attention = F.softmax(energy,dim=1)
        context = torch.bmm(attention.unsqueeze(1),value).squeeze(1)

        return context, attention
        # we return attention weights for plotting (for debugging)


In [ ]:
class MultiHeadAttention(nn.Module):
    '''
    5. Multi-Head Attention
        # Check "attention is all you need" Section 3.2.2
        h = Number of heads
        W_Q, W_K, W_V: Weight matrix for Q, K, V (h of them in total)
        W_O: d_v -> d_v

        Reshape K: (B, T, d_k)
        to (B, T, h, d_k // h) and transpose to (B, h, T, d_k // h)
        Reshape V: (B, T, d_v)
        to (B, T, h, d_v // h) and transpose to (B, h, T, d_v // h)
        Reshape Q: (B, d_q)
        to (B, h, d_q // h)

        energy = Q @ K^T
        energy = mask(energy)
        attention = softmax(energy)
        multi_head = attention @ V
        multi_head = multi_head reshaped to (B, d_v)
        context = multi_head @ W_O
    '''
    def __init__(self,embed_dim, d_qkv, head):
        "Take in model size (embedding dim) and number of heads"
        super(MultiHeadAttention, self).__init__()
        self.head_dim = d_qkv // head
        self.head = head
        self.W_Q = nn.Linear(d_qkv, head * self.head_dim)
        self.W_K = nn.Linear(d_qkv, head * self.head_dim)
        self.W_V = nn.Linear(d_qkv, head * self.head_dim)
        self.W_O = nn.Linear(d_qkv, d_qkv)

    def linear_transform(self,key,value):
        """
        This is to used in muti-head attention to avoid repeated calculation
        """
        key = self.W_K(key)
        value = self.W_V(value)
        return key,value



    def forward(self, query, key, value, mask):
        """
        input:
            key: (batch_size, seq_len, d_k)
            value: (batch_size, seq_len, d_v)
            query: (batch_size, d_q)
        * Hint: d_k == d_v == d_q is often true if you use linear projections
        return:
            context: (batch_size, key_val_dim)
        
        """
        
        h = self.head
        B, T, d_k = key.shape
        key = key.reshape(B, T, h, d_k // h).permute(0,2,1,3)
        
        B, T, d_v = value.shape
        value = value.reshape(B, T, h, d_v // h).permute(0,2,1,3)
        
        B, d_q = query.shape        
        query = self.W_Q(query)
        query = query.reshape(B, h, d_q // h)

        assert d_k == d_v == d_q

        mask = mask.cuda()

        # energy = Q @ K^T
        # (B, h, d_q // h) mat-mat product (B, h, T, d_k // h)
        """
        torch.Size([64, 2, 1, 128])
        torch.Size([64, 2, 128, 206])
        """
        energy = query.unsqueeze(2)@key.permute(0,1,3,2)
        energy = energy.squeeze(2).cuda()
        energy = energy.masked_fill(mask.unsqueeze(1),float("-inf"))
        attention = F.softmax(energy,dim=-1)

        '''
        multi_head = attention @ V
        multi_head = multi_head reshaped to (B, d_v)
        context = multi_head @ W_O

        attention: torch.Size([64, 2, 203])
        value: torch.Size([64, 2, 203, 128])
        '''

        multi_head = attention.unsqueeze(2) @ value
        multi_head = multi_head.squeeze(2).reshape(B,-1)
        context = self.W_O(multi_head)

        return context, attention
        # we return attention weights for plotting (for debugging)


# Decoding 

In [ ]:
class Decoder(nn.Module):
    '''
    As mentioned in a previous recitation, each forward call of decoder deals with just one time step.
    Thus we use LSTMCell instead of LSTM here.
    The output from the last LSTMCell can be used as a query for calculating attention.
    Methods like Gumble noise and teacher forcing can also be incorporated for improving the performance.
    '''
    def __init__(self, vocab_size, decoder_hidden_dim, embed_dim, key_value_size=128):
        super(Decoder, self).__init__()
        # Hint: Be careful with the padding_idx
        self.embedding = nn.Embedding(vocab_size,embed_dim,padding_idx=letter2index['<eos>'])

        # The number of cells is defined based on the paper
        self.lstm1 = nn.LSTMCell(embed_dim+key_value_size,decoder_hidden_dim)
        self.lstm2 = nn.LSTMCell(decoder_hidden_dim, key_value_size)
        
        self.fc1 = nn.Linear(2 * key_value_size, 4 * key_value_size)
        self.tanh1 = nn.Tanh()
        self.fc2 = nn.Linear(4 * key_value_size, embed_dim)
        self.tanh2 = nn.Tanh()

        self.dropout = nn.Dropout(0.2)
        
        # self.attention = Attention()     
        self.multihead_attention =  MultiHeadAttention(embed_dim,key_value_size,2)   
        # self.multihead_attention = nn.MultiheadAttention(embed_dim,2)     

        self.vocab_size = vocab_size
        # Optional: Weight-tying
        self.character_prob = nn.Linear(2 * key_value_size, vocab_size) #: d_v -> vocab_size
        self.key_value_size = key_value_size
        
        # Weight tying
        self.character_prob.weight = self.embedding.weight

        self._init_weights()

    # This initilization methods was referenced from of the github repository, but I could not find the link when
    # I plan to reference it. The first embedding and linear initlization was same as the paper in HW4 suggested,
    # but the init methods in LSTM is new to me. I evaluated the performance between my version with this version,
    # the remaining version could achieve higher performance.
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Embedding):
                nn.init.uniform_(m.weight, -1, 1)
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.constant_(m.bias, 0.01)	
            if isinstance(m, nn.LSTM):
                for param in m.parameters():
                    if len(param.shape) >= 2:
                        nn.init.orthogonal_(param.data)
                    else:
                        nn.init.normal_(param.data)
    
    # def _init_weights(self,hidden_dim):
    #     for m in self.modules():
    #       if isinstance(m,nn.Embedding):
    #           nn.init.uniform_(m.weight, -0.1, 0.1)
    #       if isinstance(m, nn.Linear):
    #           nn.init.xavier_normal_(m.weight)
    #           nn.init.constant_(m.bias, 0.01)	
    #       if isinstance(m, nn.LSTM):
    #           nn.init.uniform_(m.weight,-1/np.sqrt(hidden_dim),1/np.sqrt(hidden_dim))


    def forward(self, key, value, encoder_len, y=None, mode='train',Teacher_forcing_rate=0.9):
        '''
        Args:
            key :(B, T, d_k) - Output of the Encoder (possibly from the Key projection layer)
            value: (B, T, d_v) - Output of the Encoder (possibly from the Value projection layer) [128, 205, 128]
            y: (B, text_len) - Batch input of text with text_length
            mode: Train or eval mode for teacher forcing
        Return:
            predictions: the character perdiction probability 
        '''

        B, T, d_k = key.shape

        if mode == 'train':
            max_len =  y.shape[1]
            char_embeddings = self.embedding(y.long())
            # print("char_embedding shape",char_embeddings.size()) #(128,307,56)
        else:
            max_len = 600

        # TODO: Create the attention mask here (outside the for loop rather than inside) to aviod repetition 
        # generate mask
        mask = encoder_len.unsqueeze(1)< torch.arange(T).unsqueeze(0)# encoder_len (L,1) < sequence length (1,N) 
        mask = mask.cuda() 
        # print("mask size",mask.size()) # ([128, 205])
        
        predictions = []
        # This is the first input to the decoder
        # What should the fill_value be? => <SOS> 0 
        prediction = torch.full((B,1), fill_value=0, device=device) # (batch,1)
        # print("prediction's shape",prediction.shape)

        # The length of hidden_states vector should depend on the number of LSTM Cells defined in init
        # The paper uses 2
        hidden_states = [None, None] 
        
        # TODO: Initialize the context
        context =  value[:,0,:].squeeze(1) # (B, T, d_v)
        # print("value shape",value.size()) #[128, 205, 128]
        # print("context's shape",context.size())#([128, 128])

        attention_plot = [] # this is for debugging


        for i in range(max_len):
            if mode == 'train':
                # TODO: Implement Teacher Forcing 
                """
                if using teacher_forcing:
                    if i == 0:
                        # This is the first time step
                        # Hint: How did you initialize "prediction" variable above?
                    else:
                        # Otherwise, feed the label of the **previous** time step
                else:
                    char_embed = embedding of the previous prediction
                """   

                if np.random.random_sample() < Teacher_forcing_rate and i > 0:
                  # just direct copy it from the previous embeddings
                    char_embed = char_embeddings[:,i-1]
                else:
                  # This is the first time step, it is just filled with <SOS>
                    char_embed = self.embedding(prediction.argmax(dim=-1))
            else:
                # mode = evaluation, no teaching force required
                # embedding of the previous predicitons
                char_embed = self.embedding(prediction.argmax(dim=-1))

            # print("char embedding size",char_embed.size()) # (128,256)
            # print("context size",context.size()) #[128, 128])

            # what vectors should be concatenated as a context?
            y_context = torch.cat([char_embed, context], dim=1)
            # context and hidden states of lstm 1 from the previous time step should be fed
            hidden_states[0] = self.lstm1(y_context, hidden_states[0])
            # hidden states of lstm1 and hidden states of lstm2 from the previous time step should be fed
            hidden_states[1] = self.lstm2(hidden_states[0][0], hidden_states[1])
           
            # What then is the query?
            query = hidden_states[1][0]
            
            # Compute attention from the output of the second LSTM Cell
            # context, attention = self.attention(query, key, value, mask)
            
            # Implement the multi-head attention:
            """
            print(query.shape) torch.Size([64, 256]) # (B,L,Eq)
            print(key.shape)   torch.Size([64, 204, 256]) # (B,N,Ek)
            print(value.shape) torch.Size([64, 204, 256]) # (B,L,Ev)
            print(mask.shape)  torch.Size([64, 204]) # 
            context, attention = self.multihead_attention(query.unsqueeze(1), key, value, attn_mask = mask)
            """
            context, attention = self.multihead_attention(query, key, value, mask)

            # We store the first attention of this batch for debugging
            attention_plot.append(attention[0].detach().cpu())
            
            # What should be concatenated as the output context?
            output_context = torch.cat([query,context], dim=1)
            
            output_context = self.fc1(output_context)
            output_context = self.tanh1(output_context)
            # output_context = self.dropout(output_context)
            output_context = self.fc2(output_context)
            output_context = self.tanh2(output_context)
            # output_context = self.dropout(output_context)
            
            prediction = self.character_prob(output_context)
            # store predictions
            predictions.append(prediction.unsqueeze(1))
        
        # Concatenate the attention and predictions to return
        attentions = torch.stack(attention_plot, dim=0)
        predictions = torch.cat(predictions, dim=1)
        return predictions, attentions

In [ ]:
class Seq2Seq(nn.Module):
    '''
    We train an end-to-end sequence to sequence model comprising of Encoder and Decoder.
    This is simply a wrapper "model" for your encoder and decoder.
    '''
    def __init__(self, input_dim, vocab_size, encoder_hidden_dim, decoder_hidden_dim, embed_dim, key_value_size=128):
        super(Seq2Seq,self).__init__()
        self.encoder = Encoder(input_dim, encoder_hidden_dim, key_value_size=key_value_size)
        self.decoder = Decoder(vocab_size, decoder_hidden_dim, embed_dim, key_value_size=key_value_size)
        
    def forward(self, x, x_len, y=None, mode='train', tf_rate=0.9):
        x = T.FrequencyMasking(freq_mask_param=2)(x)
        key, value, encoder_len = self.encoder(x, x_len)
        key, value = self.decoder.multihead_attention.linear_transform(key,value)
        predictions, attentions = self.decoder(key, value, encoder_len, y=y, mode=mode,Teacher_forcing_rate=tf_rate)
        return predictions, attentions

In [ ]:
model = Seq2Seq(input_dim=13, vocab_size=len(LETTER_LIST), encoder_hidden_dim=256, decoder_hidden_dim=512, embed_dim=256, key_value_size=256)

model = model.to(device)

# print(model)
# summary(encoder, x.to(device), lx) 

# Training

In [ ]:
# optimizer = optim.Adam(model.parameters(), lr=lr)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.75, patience=5)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=0.01)
num_steps = len(train_loader) * n_epochs
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_steps)
warmup_scheduler = warmup.RAdamWarmup(optimizer)
criterion = nn.CrossEntropyLoss(reduction='none').to(device)
scaler = torch.cuda.amp.GradScaler()

In [ ]:
def train(model, train_loader, criterion, optimizer, mode, tf_rate,warmup_scheduler,lr_scheduler):
    model.train()
    running_loss = 0
    attentions =None
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

    # 0) Iterate through your data loader
    for i, (x, y, lx, ly) in enumerate(train_loader):
        optimizer.zero_grad()
        # 1) Send the inputs to the device
        x,y = x.cuda(),y.long().cuda()
        # print(x)
        # print(y)

        # 2) Pass your inputs, and length of speech into the model.
        with torch.cuda.amp.autocast():  
            predictions, attentions = model(x, lx, y, mode=mode,tf_rate=tf_rate)
        
            # 3) Generate a mask based on target length. This is to mark padded elements
            # so that we can exclude them from computing loss.
            # Ensure that the mask is on the device and is the correct shape.
            mask =torch.zeros(y.size()).T.cuda()
            for idx, length_Y in enumerate(ly):
                mask[:length_Y,idx] = 1
                
            # 4) Make sure you have the correct shape of predictions when putting into criterion
            loss = criterion(predictions.view(-1, predictions.size(2)), y.view(-1)) 
            # Use the mask you defined above to compute the average loss
            masked_loss = torch.sum(loss * mask.reshape(-1)) / torch.sum(mask)
        
        running_loss += masked_loss.item()

        batch_bar.set_postfix(
                loss="{:.04f}".format(float(masked_loss)),
                lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
        lr_wb = round(float(optimizer.param_groups[0]['lr']),8)

        # 5) backprop
        scaler.scale(masked_loss).backward()
        scaler.step(optimizer)
        scaler.update()
        with warmup_scheduler.dampening():
            lr_scheduler.step()

        # Optional: Gradient clipping
        # When computing Levenshtein distance, make sure you truncate prediction/target

        batch_bar.update()
    batch_bar.close()

    avg_loss = running_loss / len(train_loader)
    del x,y,lx, ly
    torch.cuda.empty_cache()
    return avg_loss, lr_wb

        # Optional: plot your attention for debugging
    # plot_attention(attentions)
        
def val(model, valid_loader):
    model.eval()
    model.cuda()
    running_dist = 0
    # running_loss = 0
    total_num = 0

    with torch.no_grad():
        for batch_idx, (x, y, lx, ly) in enumerate(valid_loader):
            x, y= x.cuda(),y.long().cuda()
            predictions, _ = model(x, lx, y=y, mode='eval')
            
            # text prediction
            pred_text = transform_index_to_letter(predictions.argmax(-1).detach().cpu().numpy())    
            target_text =transform_index_to_letter(y.detach().cpu().numpy())

            for pred, target in zip(pred_text, target_text):
                dist = lev.distance(pred, target)
                running_dist += dist
                total_num += 1
    
    avg_dist_eval = running_dist / total_num
    # avg_loss_eval = running_loss / len(valid_loader)

    del x,y,lx,ly
    torch.cuda.empty_cache()
    return  avg_dist_eval
    # return avg_loss_eval, avg_dist_eval

In [ ]:
# TODO: Define your model and put it on the device here
# ...
torch.cuda.empty_cache()
nn.CrossEntropyLoss(reduction='none').to(device)
# Make sure you understand the implication of setting reduction = 'none'
criterion = nn.CrossEntropyLoss(reduction='none')
mode = 'train'

for epoch in range(n_epochs):
    avg_loss, lr_wb = train(model, train_loader, criterion, optimizer, mode,tf_rate=tf,warmup_scheduler=warmup_scheduler,lr_scheduler=scheduler)
    print(f'Epoch {epoch}: Avg-Loss: {avg_loss}, lr: {lr_wb}')

    avg_dist_eval = val(model, val_loader)
    print(f'Validation: Avg-distance: {avg_dist_eval}')

    if epoch % tf_sep_epoch == 0:
        tf = tf - 0.05
        tf = max(0.6,tf)
    
    wandb.log({"Avg-distance:":avg_dist_eval,"Training avg loss":avg_loss, "lr":lr_wb, "tf":tf})


    path = data_path+f"/model_epoch_{epoch}.ckpt"
    torch.save({'epoch': epoch,'model_state_dict': model.state_dict(),'optimizer_state_dict': optimizer.state_dict()}, path)

Train:   0%|          | 0/223 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').


Epoch 0: Avg-Loss: 1.8926060376145915, lr: 0.00032641
Validation: Avg-distance: 555.0192378838328


Epoch 1: Avg-Loss: 1.4430679050796236, lr: 0.00045332
Validation: Avg-distance: 534.457269700333


Epoch 2: Avg-Loss: 1.2996605132192773, lr: 0.00054383
Validation: Avg-distance: 534.2571217166112


Epoch 3: Avg-Loss: 1.2294078512042093, lr: 0.00061424
Validation: Avg-distance: 534.0828708842027


...

Epoch 95: Avg-Loss: 0.0022693722941620367, lr: 3.95e-06
Validation: Avg-distance: 8.65778764335923


Epoch 96: Avg-Loss: 0.0021108175187283967, lr: 2.23e-06
Validation: Avg-distance: 8.642619311875693


Epoch 97: Avg-Loss: 0.0022211883657177567, lr: 9.9e-07
Validation: Avg-distance: 8.619311875693674


Epoch 98: Avg-Loss: 0.002021765347096167, lr: 2.5e-07
Validation: Avg-distance: 8.620421753607102


Epoch 99: Avg-Loss: 0.0020278023246304037, lr: 0.0
Validation: Avg-distance: 8.608953015168332


In [ ]:
path

'/content/drive/MyDrive/hw4/store_checkpoints/Freq_Aug_2_Multiheaded_Masking_AdamW_SameKeyValueIncreseTest_Cepstral_tf_0.9_sep5_embedding_init_warmup_radam_lockeddropout_0.2/model_epoch_99.ckpt'

In [ ]:
# path = "/content/drive/MyDrive/hw4/store_checkpoints/KeyValueIncreseTest_Cepstral_tf_0.9_sep5_embedding_init_warmup_radam_lockeddropout_0.2/model_epoch_87.ckpt"
# checkpoint = torch.load(path)
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Submission

In [ ]:
def testing(model, test_loader):
    model.eval()
    model.to(device)
    pred_text_cat = []
    with torch.no_grad():
        for batch_idx, (x,lx) in enumerate(test_loader):
            x = x.to(device)
            predictions,_ = model(x, lx, mode='eval')
            pred_text = transform_index_to_letter(predictions.argmax(-1).detach().cpu().numpy())
            # print(predText)    
            pred_text_cat.extend(pred_text)

    del x,lx
    torch.cuda.empty_cache()
    return pred_text_cat

res = testing(model, test_loader)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').


In [ ]:
# write csv file
import pandas as pd
id = np.array(range(len(res)))
predictions = res
df = pd.DataFrame({"id":id,"predictions":predictions})
df.to_csv("submission.csv",index=False)

!kaggle competitions submit -c 11-785-s22-hw4p2 -f submission.csv -m "Message"

100% 288k/288k [00:00<00:00, 1.48MB/s]
Successfully submitted to Attention-Based Speech Recognition

In [ ]:
predictions

In [ ]:
'''
Debugging suggestions from Eason, a TA from previous semesters:

(1) Decrease your batch_size to 2 and print out the value and shape of all intermediate variables to check if they satisfy the expectation
(2) Be super careful about the LR, don't make it too high. Too large LR would lead to divergence and your attention plot will never make sense
(3) Make sure you have correctly handled the situation for time_step = 0 when teacher forcing

(1) is super important and is the most efficient way for debugging. 
'''
'''
Tips for passing A from B (from easy to hard):
** You need to implement all of these yourself without utilizing any library **
(1) Increase model capacity. E.g. increase num_layer of lstm
(2) LR and Teacher Forcing are also very important, you can tune them or their scheduler as well. Do NOT change lr or tf during the warm-up stage!
(3) Weight tying
(4) Locked Dropout - insert between the plstm layers
(5) Pre-training decoder or train an LM to help make predictions
(5) Pre-training decoder to speed up the convergence: 
    disable your encoder and only train the decoder like train a language model
(6) Better weight initialization technique
(7) Batch Norm between plstm. You definitely can try other positions as well
(8) Data Augmentation. Time-masking, frequency masking
(9) Weight smoothing (avg the last few epoch's weight)
(10) You can try CNN + Maxpooling (Avg). Some students replace the entire plstm blocks with it and some just combine them together.
(11) Beam Search
'''

In [ ]:
letter2index

# Toy Data for debugging

In [ ]:
%cd /hw4p2_simple/hw4p2_simple
# Load the training, validation and testing data
train_data = np.load('train.npy', allow_pickle=True, encoding='bytes')
valid_data = np.load('dev.npy', allow_pickle=True, encoding='bytes')

# Load the training, validation raw text transcripts
raw_train_transcript = np.load('train_transcripts.npy', allow_pickle=True,encoding='bytes')
raw_valid_transcript = np.load('dev_transcripts.npy', allow_pickle=True,encoding='bytes')

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, index):
        # For training and validation set, return x and y
        X = torch.tensor(self.X[index])
        # print(self.Y[index])
        Y = [letter2index[yy] for yy in self.Y[index]]
        Yy = torch.LongTensor(Y)
        return X, Yy
    
    def collate_fn(batch):
        batch_x = [x for (x, y) in batch]
        batch_y = [y for (x, y) in batch]

        batch_x_pad = pad_sequence([torch.Tensor(x) for x in batch_x],batch_first=True)  # TODO: pad the sequence with pad_sequence (already imported)
        lengths_x = torch.LongTensor([len(x) for x in batch_x])  # TODO: Get original lengths of the sequence before padding

        batch_y_pad = pad_sequence(batch_y,batch_first=True,padding_value=0)  # TODO: pad the sequence with pad_sequence (already imported)
        lengths_y = torch.LongTensor([len(y) for y in batch_y])   # TODO: Get original lengths of the sequence before padding

        return batch_x_pad, batch_y_pad, torch.tensor(lengths_x), torch.tensor(lengths_y)

# Create datasets
train_dataset = MyDataset(train_data,raw_train_transcript)
valid_dataset = MyDataset(valid_data,raw_valid_transcript)

# Create data loaders
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=64, collate_fn = MyDataset.collate_fn,num_workers=0, pin_memory=True)
val_loader = DataLoader(valid_dataset, shuffle=False, batch_size=128, collate_fn = MyDataset.collate_fn,num_workers=0, pin_memory=True)


In [ ]:
def train(model, train_loader, criterion, optimizer, mode):
    model.train()
    running_loss = 0
    attentions = None
    
    # 0) Iterate through your data loader
    for i, (x, y, lx, ly) in enumerate(train_loader):
        optimizer.zero_grad()
        # 1) Send the inputs to the device
        x,y = x.cuda(),y.long().cuda()
        # print(x)
        # print(y)

        # 2) Pass your inputs, and length of speech into the model.
        predictions, attentions = model(x, lx, y, mode=mode)
        
        # 3) Generate a mask based on target length. This is to mark padded elements
        # so that we can exclude them from computing loss.
        # Ensure that the mask is on the device and is the correct shape.
        mask =torch.zeros(y.size()).T.to(device)
        for idx, length_Y in enumerate(ly):
            mask[:length_Y,idx] = 1
            
        # 4) Make sure you have the correct shape of predictions when putting into criterion
        loss =  criterion(predictions.view(-1, predictions.size(2)), y.view(-1)) 
        # Use the mask you defined above to compute the average loss
        masked_loss = torch.sum(loss * mask.reshape(-1)) / torch.sum(mask)

        # 5) backprop
        masked_loss.backward()
        # Optional: Gradient clipping

        # When computing Levenshtein distance, make sure you truncate prediction/target
        optimizer.step()

        # Optional: plot your attention for debugging
    plot_attention(attentions)
        
def val(model, valid_loader):
    model.eval()
    model.to(device)
    runningDist = 0
    tot_sec = 0
    with torch.no_grad():
        for batch_idx, (x,y,x_len,y_len) in tqdm(enumerate(valid_loader),position=0, leave=True):
            x,y,x_len,y_len = x.to(device),y.long().to(device),x_len.to(device),y_len.to(device)
            predictions,_ = model(x, x_len, y=y, mode='eval')           

            predText = transform_index_to_letter(predictions.argmax(-1).detach().cpu().numpy())    
            targetText =transform_index_to_letter(y.detach().cpu().numpy())

            print("=============predText=================")
            print(predText)
            print("y",y)
            print("y.shape",y.shape) # (100,36)
            print("=============predText=================")

            # print("=============TargetText=================")
            # print(targetText)
            # print("=============TargetText=================")
            for pred, target in zip(predText, targetText):
                dist = lev.distance(pred, target)
                runningDist += dist
                tot_sec += 1

    print('eval:')
    print('Avg-distance: {:.5f}'.format(runningDist / tot_sec))
    del x,y,x_len,y_len
    torch.cuda.empty_cache()
    return runningDist / tot_sec

def transform_index_to_letter(batch_indices):
    '''
    Transforms numerical index input to string output by converting each index 
    to its corresponding letter from LETTER_LIST

    Args:
        batch_indices: List of indices from LETTER_LIST with the shape of (N, )
    
    Return:
        transcripts: List of converted string transcripts. This would be a list with a length of N
    '''
    transcripts = []
    start_index = letter2index['<sos>']
    stop_index = letter2index['<eos>']

    print("batch indices",batch_indices)

    for tr in batch_indices:
      curr = ""
      for i in tr:
        if i == stop_index:
          break
        elif i == start_index:
          pass
        else:
          curr += index2letter[i]
      transcripts.append(curr)
    
    print(transcripts)


In [ ]:
# TODO: Define your model and put it on the device here
# ...
torch.cuda.empty_cache()
n_epochs = 20
nn.CrossEntropyLoss(reduction='none').to(device)
# Make sure you understand the implication of setting reduction = 'none'
criterion = nn.CrossEntropyLoss(reduction='none')
mode = 'train'

for epoch in range(n_epochs):
    train(model, train_loader, criterion, optimizer, mode)
    val(model, val_loader)